In [ ]:
# step1_load_base_model.py
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
import torch

model_name = "LiquidAI/LFM2-350M"  # Small model for testing

print("Loading base model...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,
    device_map="cuda"
)

print(f"Model loaded! It has {base_model.num_parameters():,} parameters")

# Test it
prompt = "Write a Python function to add two numbers:"
inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)
outputs = base_model.generate(**inputs, max_length=100)
print("\nBase model output:")
print(tokenizer.decode(outputs, skip_special_tokens=True))

In [ ]:
# step2_prepare_data.py
from datasets import Dataset

# Example: Train it to write Python code
data = [
    {
        "instruction": "Write a function to reverse a string",
        "output": "def reverse_string(s):\n    return s[::-1]"
    },
    {
        "instruction": "Write a function to check if a number is even",
        "output": "def is_even(n):\n    return n % 2 == 0"
    },
    {
        "instruction": "Write a function to find the maximum in a list",
        "output": "def find_max(lst):\n    return max(lst)"
    },
    # Add more examples...
]

# Convert to dataset
dataset = Dataset.from_list(data)

# Format for training
def format_instruction(sample):
    return f"### Instruction:\n{sample['instruction']}\n\n### Response:\n{sample['output']}"

# Save it
dataset.to_json("training_data.json")
print(f"Created dataset with {len(dataset)} examples")

In [ ]:
data = [
    {
        "instruction": "Write a haiku about coding",
        "output": "Silent loops awaken\nLogic hums in midnight glow\nBugs fade into dawn"
    },
    {
        "instruction": "Explain recursion in simple terms",
        "output": "Recursion is when a function solves a problem by calling itself on smaller versions of the same problem until it reaches a stopping condition."
    },
    {
        "instruction": "Write a short story about a lonely robot",
        "output": "The robot powered on each night in the empty factory. With no humans left, it replayed old voice logs, pretending someone was still there to listen."
    },
    {
        "instruction": "Generate JSON to open the camera app",
        "output": "{\"action\": \"open_app\", \"app_name\": \"camera\"}"
    },
    {
        "instruction": "Generate JSON to send a WhatsApp message to Rahul saying Hello",
        "output": "{\"action\": \"send_message\", \"app_name\": \"whatsapp\", \"contact_name\": \"Rahul\", \"message\": \"Hello\"}"
    },
    {
        "instruction": "Summarize what an API is",
        "output": "An API is a set of rules that allows different software systems to communicate and exchange data with each other."
    },
    {
        "instruction": "Write a motivational quote about learning",
        "output": "Learning is the quiet investment that compounds into confidence."
    },
    {
        "instruction": "Generate JSON to enable WiFi",
        "output": "{\"action\": \"toggle_setting\", \"setting\": \"wifi\", \"state\": \"on\"}"
    },
    {
        "instruction": "Explain what a database is",
        "output": "A database is an organized collection of data that can be stored, managed, and retrieved efficiently."
    },
    {
        "instruction": "Write a funny one-line joke about programmers",
        "output": "Programmers don’t get lost, they just take unexpected iterations."
    },
    {
        "instruction": "Generate JSON to play music",
        "output": "{\"action\": \"play_music\"}"
    },
    {
        "instruction": "Describe artificial intelligence in one sentence",
        "output": "Artificial intelligence is the simulation of human-like reasoning and learning in machines."
    },
    {
        "instruction": "Generate JSON to set an alarm for 7 AM",
        "output": "{\"action\": \"set_alarm\", \"time\": \"07:00\"}"
    },
    {
        "instruction": "Explain machine learning briefly",
        "output": "Machine learning is a method where computers learn patterns from data to make predictions or decisions without being explicitly programmed for each task."
    },
    {
        "instruction": "Write a two-line poem about space",
        "output": "Stars whisper ancient light,\nTime drifts across the dark."
    },
    {
        "instruction": "Generate JSON to open YouTube",
        "output": "{\"action\": \"open_app\", \"app_name\": \"youtube\"}"
    },
    {
        "instruction": "What is cloud computing?",
        "output": "Cloud computing is the delivery of computing services like storage, servers, and software over the internet instead of local machines."
    },
    {
        "instruction": "Generate JSON to increase volume",
        "output": "{\"action\": \"adjust_volume\", \"direction\": \"up\"}"
    },
    {
        "instruction": "Write a short description of blockchain",
        "output": "Blockchain is a distributed ledger technology that records transactions securely across multiple computers."
    },
    {
        "instruction": "Generate JSON to turn on Bluetooth",
        "output": "{\"action\": \"toggle_setting\", \"setting\": \"bluetooth\", \"state\": \"on\"}"
    }
]
dataset = Dataset.from_list(data)



In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()  # Shows how few params we're training!

In [ ]:
from datasets import load_dataset

#dataset = load_dataset("json", data_files="training_data.json", split="train")

def format_data(sample):
    text = f"### Instruction:\n{sample['instruction']}\n\n### Response:\n{sample['output']}"
    return {"text": text}

dataset = dataset.map(format_data)

# 5. Tokenize
def tokenize(sample):
    result = tokenizer(
        sample["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)


In [ ]:
# 6. Training arguments
training_args = TrainingArguments(
    output_dir="./lora_adapter_coding",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
)

# 7. Train!
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

print("Starting training...")
trainer.train()

# 8. Save the adapter
model.save_pretrained("./lora_adapter_coding")
tokenizer.save_pretrained("./lora_adapter_coding")
print("LoRA adapter saved to ./lora_adapter_coding")

In [ ]:

# 6. Training arguments
training_args = TrainingArguments(
    output_dir="./lora_adapter_creative",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
)

# 7. Train!
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

print("Starting training...")
trainer.train()

# 8. Save the adapter
model.save_pretrained("./lora_adapter_creative")
tokenizer.save_pretrained("./lora_adapter_creative")
print("LoRA adapter saved to ./lora_adapter_creative")

In [ ]:
# step5_switch_adapters.py
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# 2. Load FIRST adapter (coding)
print("=" * 50)
print("Loading CODING adapter...")
model = PeftModel.from_pretrained(base_model, "./lora_adapter_coding")

prompt = "### Instruction:\nWrite a function to calculate factorial\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=150, temperature=0.7)
print("CODING ADAPTER OUTPUT:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# 3. SWAP to second adapter (creative)
print("\n" + "=" * 50)
print("SWAPPING to CREATIVE adapter...")
model.load_adapter("./lora_adapter_creative", adapter_name="creative")
model.set_adapter("creative")

prompt = "### Instruction:\nWrite a poem about AI\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=150, temperature=0.7)
print("CREATIVE ADAPTER OUTPUT:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# 4. Switch BACK to coding
print("\n" + "=" * 50)
print("SWAPPING back to CODING adapter...")
model.set_adapter("default")  # "default" is the first loaded adapter

prompt = "### Instruction:\nWrite a function to sort a list\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=150, temperature=0.7)
print("CODING ADAPTER OUTPUT (again):")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

print("\n" + "=" * 50)
print("Notice: Base model stayed in memory the whole time!")
print("We just swapped tiny adapter files (~20MB each)")